In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import time
import random
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [2]:
"""# Load data
data = np.load("data/data_1.npy")
labels = np.load("data/labels_1.npy")

data = np.vstack((data,np.load("data/data_2.npy")))
labels = np.vstack((labels,np.load("data/labels_2.npy")))

data = np.vstack((data,np.load("data/data_3.npy")))
labels = np.vstack((labels,np.load("data/labels_3.npy")))

data = np.vstack((data,np.load("data/data_4.npy")))
labels = np.vstack((labels,np.load("data/labels_4.npy")))

data = np.vstack((data,np.load("data/data_5.npy")))
labels = np.vstack((labels,np.load("data/labels_5.npy")))"""

data = np.load("../data/images.npy")/255.
data = torch.FloatTensor(np.expand_dims(data,axis=1))
data_permuts = torch.randperm(data.shape[0])
data = data[data_permuts,:]
labels = torch.FloatTensor(np.load("../data/labels.npy"))
labels = labels[data_permuts,:]

print(f"data: {data.shape}, labels: {labels.shape}")

data: torch.Size([18000, 1, 150, 150]), labels: torch.Size([18000, 2])


<br/>
<h3>Labels transformation to cyclical</h3>

In [3]:
#we will use a dataframe to change the labels since it is more convenient.
labels_df = pd.DataFrame(labels,columns=['hour', 'minute'])
labels_df['h_cos'] = np.cos(2 * np.pi * labels_df["hour"] / labels_df["hour"].max())
labels_df['h_sin'] = np.sin(2 * np.pi * labels_df["hour"] / labels_df["hour"].max())
labels_df['m_cos'] = np.cos(2 * np.pi * labels_df["minute"] / labels_df["minute"].max())
labels_df['m_sin'] = np.sin(2 * np.pi * labels_df["minute"] / labels_df["minute"].max())

In [4]:
labels_df

,hour,minute,h_cos,h_sin,m_cos,m_sin
0,5.0,34.0,-0.959493,2.817324e-01,-0.887352,-0.461093
1,9.0,35.0,0.415415,-9.096320e-01,-0.833314,-0.552800
2,1.0,32.0,0.841254,5.406408e-01,-0.964768,-0.263103
3,11.0,8.0,1.000000,6.516827e-07,0.658511,0.752571
4,7.0,4.0,-0.654861,-7.557498e-01,0.910635,0.413212
...,...,...,...,...,...,...
17995,0.0,48.0,1.000000,0.000000e+00,0.388824,-0.921312
17996,3.0,15.0,-0.142315,9.898214e-01,-0.026621,0.999646
17997,5.0,19.0,-0.959493,2.817324e-01,-0.437307,0.899312
17998,7.0,31.0,-0.654861,-7.557498e-01,-0.987268,-0.159063


In [5]:
labels_cycl = torch.FloatTensor(labels_df.to_numpy())

In [6]:
labels_cycl

tensor([[ 5.0000e+00,  3.4000e+01, -9.5949e-01,  2.8173e-01, -8.8735e-01,
         -4.6109e-01],
        [ 9.0000e+00,  3.5000e+01,  4.1542e-01, -9.0963e-01, -8.3331e-01,
         -5.5280e-01],
        [ 1.0000e+00,  3.2000e+01,  8.4125e-01,  5.4064e-01, -9.6477e-01,
         -2.6310e-01],
        ...,
        [ 5.0000e+00,  1.9000e+01, -9.5949e-01,  2.8173e-01, -4.3731e-01,
          8.9931e-01],
        [ 7.0000e+00,  3.1000e+01, -6.5486e-01, -7.5575e-01, -9.8727e-01,
         -1.5906e-01],
        [ 5.0000e+00,  4.4000e+01, -9.5949e-01,  2.8173e-01, -2.6620e-02,
         -9.9965e-01]])

<br/>
<h3>Dataset creation</h3>

In [7]:
class ClockDataset(Dataset):
    """ Class used to create the pytorch DataLoaders.
    """
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y
    
    def __len__(self):
        return len(self.data)

In [8]:
# Create main dataset
clock_dataset = ClockDataset(data, labels_cycl)
train_data, test_data, val_data = random_split(clock_dataset, [14000,3000,1000])

# Split dataset into train, test and validation sets
train_data = DataLoader(train_data, batch_size=64)
test_data = DataLoader(test_data, batch_size=64)
val_data = DataLoader(val_data, batch_size=64)

In [9]:
for X, y in train_data:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 150, 150])
Shape of y: torch.Size([64, 6]) torch.float32


<br/>
<h3>Model creation</h3>

In [10]:
class NN_regression(nn.Module):
    """ Convolution model that returns one value as output.
    """
    def __init__(self, input_channels, h, w, n_outputs):
        super(NN_regression, self).__init__()

        self.input_layer = nn.Sequential(
            nn.Conv2d(input_channels, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(16)
        )
        self.hidden_layers = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=2),
            nn.Flatten()
        )


        self.output_layer = nn.Sequential(
            nn.Linear(32768, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, n_outputs)
            
        )


    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x

In [11]:
model = NN_regression(input_channels=1,h=150,w=150, n_outputs=4).to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    losses = []
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y[:, 2:].to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss)
    losses = torch.FloatTensor(losses)
    print(f"Train loss: {torch.mean(losses):>7f}")

In [13]:
def test(dataloader, model, loss_fn):
    model.eval()
    losses = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y[:, 2:].to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            losses.append(loss)
    losses = torch.FloatTensor(losses)
    print(f"Test avg loss: {torch.mean(losses):>8f} \n")


In [14]:
epochs = 10
start_time = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data, model, loss, optimizer)
    test(val_data, model, loss)
end_time = time.time()

Epoch 1
-------------------------------
Train loss: 0.456789
Test avg loss: 0.297029 

Epoch 2
-------------------------------
Train loss: 0.244245
Test avg loss: 0.189842 

Epoch 3
-------------------------------
Train loss: 0.096588
Test avg loss: 0.069306 

Epoch 4
-------------------------------
Train loss: 0.034566
Test avg loss: 0.036245 

Epoch 5
-------------------------------
Train loss: 0.019191
Test avg loss: 0.025878 

Epoch 6
-------------------------------
Train loss: 0.013278
Test avg loss: 0.023368 

Epoch 7
-------------------------------
Train loss: 0.010957
Test avg loss: 0.025298 

Epoch 8
-------------------------------
Train loss: 0.011008
Test avg loss: 0.026835 

Epoch 9
-------------------------------
Train loss: 0.011196
Test avg loss: 0.026247 

Epoch 10
-------------------------------
Train loss: 0.011548
Test avg loss: 0.026613 



In [15]:
class CommonSenseError(nn.Module):
    def __init__(self):
        super(CommonSenseError, self).__init__()

    def forward(self, prediction, target):
        # hour common sense error in minutes
        preds = torch.floor(prediction.T)
        labels = target.T
        hour_start_idx = torch.minimum(preds[0], labels[0])
        hour_end_idx = torch.maximum(preds[0], labels[0])
        hour_dist_1 = (hour_end_idx - hour_start_idx)
        hour_dist_2 = hour_start_idx + 12 - hour_end_idx
        hour_dist = torch.minimum(hour_dist_1, hour_dist_2) * 60  # we want the common error to be in minutes

        # minutes common sense error
        mins_start_idx = torch.minimum(preds[1], labels[1])
        mins_end_idx = torch.maximum(preds[1], labels[1])
        mins_dist_1 = (mins_end_idx - mins_start_idx)
        mins_dist_2 = mins_start_idx + 60 - mins_end_idx
        mins_dist = torch.minimum(mins_dist_1, mins_dist_2)

        # return total common sense error in minutes
        return torch.mean(hour_dist + mins_dist)


a = torch.Tensor([[1,14], [4,28], [2,36], [1,1]])
b = torch.Tensor([[1,15], [3,29], [2,36], [11,59]])
cse = CommonSenseError()
cse.forward(a,b)

tensor(46.)

In [16]:
def predict(dataloader, model, loss_fn):
    model.eval()
    losses = []
    predictions = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            predictions.append(pred.cpu().numpy())
            loss = loss_fn(pred, y)
            losses.append(loss)
    losses = torch.FloatTensor(losses)
    print(f"Avg evaluation loss: {torch.mean(losses):>8f} \n")
    return predictions

In [17]:
cse = CommonSenseError()
predictions = []
with torch.no_grad():
    for curr_batch in test_data:
        X = curr_batch[0].to(device)
        pred = model(X)
        predictions.append(pred)

for pred, target in zip(predictions, test_data):
    print("preds:",pred.shape, "targets:",target[1][2:].shape)
    break

preds: torch.Size([64, 4]) targets: torch.Size([62, 6])


In [18]:
# there are some values slightly bigger than 1 and smaller than -1 so 
# we will approximate them to 1 and -1 correspectively
preds[preds > 1] = 1
preds[preds < -1] = -1

NameError: name 'preds' is not defined

In [ ]:
#returns the corresponding hour and minute predictions
#from the sine and cosine values
import math

def denormalize_clock(h_cos,h_sin,m_cos,m_sin):

    h_angle = math.atan2(h_sin, h_cos)
    h_angle *= 180 / math.pi
    if h_angle < 0: h_angle += 360

    m_angle = math.atan2(m_sin, m_cos)
    m_angle *= 180 / math.pi
    if m_angle < 0: m_angle += 360

    h_angle = math.modf(h_angle/30)[1]
    m_angle = math.modf(m_angle/6)[1]
    return [h_angle, m_angle]

In [ ]:
true_preds = np.array([ denormalize_clock(h_cos,h_sin,m_cos,m_sin) for h_cos,h_sin,m_cos,m_sin in preds])

In [ ]:
#function to calculate common sense distance in minutes between 2 values
def common_sense_distance(domain_len,value_1, value_2):
  start_index = min(value_1,value_2)
  end_index = max(value_1,value_2)
  dist_1 = end_index - start_index

  dist_2 = start_index + domain_len - end_index
  return min(dist_1,dist_2)

In [ ]:
test_data.dataset.dataset()

TypeError: 'ClockDataset' object is not callable

In [ ]:
h_domain = 12
m_domain = 60
distances= []
for i in range(3000):
  hours = common_sense_distance(h_domain,test_data_split, true_preds.T[0,i])*60
  mins = common_sense_distance(m_domain,y_test['minute'].iloc[i], true_preds.T[1,i])
  distances.append(hours+mins)

print(f"Common sense error in minutes: {np.mean(distances)}")

NameError: name 'y_test' is not defined